# Decision Support System 

## Data preperation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
sns.set_style("darkgrid"); # To show the graphs with white background and gridded
import plotly.express as px;
plt.style.use("seaborn-notebook");

In [2]:
data = pd.read_csv("../input/world-happiness-report-2021/world-happiness-report-2021.csv")

In [3]:
data

In [4]:
# Info about data
data.info()

## Data understanding 

### Happiness score in the world by regions

In [5]:
plt.figure(figsize = (15, 8));
sns.kdeplot(data["Ladder score"], hue = data["Regional indicator"], fill = True, linewidth = 2, palette = "coolwarm");
plt.axvline(data["Ladder score"].mean(), c = "black"); # Mean value of Ladder score
plt.title("Ladder score vs Regional indicators");

### Happiness score in the world by countries

In [6]:
fig = px.choropleth(data,
                    locations = "Country name",
                    color = "Ladder score",
                    locationmode = "country names"
                   );
fig.update_layout(title = "Happiness score Comparison by Countries");
fig.show();

## Data Content
#### The happiness scores and rankings use data from Gallup World Poll.
    Gallup World Poll: In 2005, Gallup began its World Poll, which continually surveys citizens in 160 countries, representing more than 98% of the world's adult population. The Gallup World Poll consists of more than 100 global questions as well as region-specific items.
#### The columns following:
    the happiness score estimate the extent to which each of six factors – economic production, social support, life expectancy, freedom, absence of corruption, and generosity – contribute to making life evaluations higher in each country than they are in Dystopia, a hypothetical country that has values equal to the world’s lowest national averages for each of the six factors. They have no impact on the total score reported for each country, but they do explain why some countries rank higher than others.

#### Ladder score:
    Happiness score or subjective well-being. This is the national average response to the question of life evaluations.

#### Logged GDP per capita:
    The GDP-per-capita time series from 2019 to 2020 using countryspecific forecasts of real GDP growth in 2020.
#### Social support: 
    Social support refers to assistance or support provided by members of social networks to an individual.
#### Healthy life expectancy:
    Healthy life expectancy is the average life in good health - that is to say without irreversible limitation of activity in daily life or incapacities - of a fictitious generation subject to the conditions of mortality and morbidity prevailing that year.
#### Freedom to make life choices: 
    Freedom to make life choices is the national average of binary responses to the GWP question “Are you satisfied or dissatisfied with your freedom to choose what you do with your life?” ... It is defined as the average of laughter and enjoyment for other waves where the happiness question was not asked
#### Generosity:
    Generosity is the residual of regressing national average of response to the GWP question “Have you donated money to a charity in the past month?” on GDP per capita. This column has negative values that won't be accepted in the algorithms of this DSS.
#### Perceptions of corruption:
    The measure is the national average of the survey responses to two questions in the GWP: “Is corruption widespread throughout the government or not” and “Is corruption widespread within businesses or not?”
#### Ladder score in Dystopia:
    It has values equal to the world’s lowest national averages. Dystopia as a benchmark against which to compare contributions from each of the six factors. Dystopia is an imaginary country that has the world's least-happy people. ... Since life would be very unpleasant in a country with the world's lowest incomes, lowest life expectancy, lowest generosity, most corruption, least freedom, and least social support, it is referred to as “Dystopia,” in contrast to Utopia
World Happiness Report Official Website: https://worldhappiness.report/

### The columns that will are kept: 
    Logged GDP per capita	Social support	Healthy life expectancy	Freedom to make life choices	Perceptions of corruption

In [7]:
#make the countries as indexes, drop the regional indicator column since it's not numerical and drop ladder score since its the score of happiness that we are searching 
df = data.copy()
df.set_index('Country name', inplace=True)
df=df.drop(["Regional indicator","Ladder score","Standard error of ladder score","upperwhisker","lowerwhisker","Ladder score in Dystopia","Explained by: Log GDP per capita","Explained by: Social support","Explained by: Healthy life expectancy","Explained by: Freedom to make life choices","Explained by: Generosity","Explained by: Perceptions of corruption","Dystopia + residual","Generosity"], axis =1)
df

## Data Distribution 

In [8]:
# Distribution of feature set 2: Logged GDP per capita
figure(figsize=(8, 6), dpi=80)
listFeatures = ["Logged GDP per capita"]
sns.boxplot(data = data.loc[:, listFeatures], orient = "v", palette = "Set2")
plt.xticks(rotation = 60)

In [9]:
# Distribution of feature set 1: Social support, Freedom to make life choices, Perceptions of corruption
figure(figsize=(8, 6), dpi=80)
listFeatures = ["Social support", "Freedom to make life choices", "Perceptions of corruption"]
sns.boxplot(data = data.loc[:, listFeatures], orient = "v", palette = "Set1")
plt.xticks(rotation = 60)

In [10]:
# Distribution of feature set 3: Healthy life expectancy
figure(figsize=(8, 6), dpi=80)
listFeatures = ["Healthy life expectancy"]
sns.boxplot(data = data.loc[:, listFeatures], orient = "v", palette = "Set3")
plt.xticks(rotation = 60)

## Data processing

## Entropy

### Step 0

In [11]:
Sum = df.sum(axis = 0)
Sum = Sum.rename("Sum") 
df0 = df.append(Sum,ignore_index=False)
df0

In [12]:
#n the number of criteria
n = df.shape[1]
n

In [13]:
# m the number of alternatives 
m = df.shape[0]
m

In [14]:
# k which is 1/ln(m)
k = 1/ np.log(m)
k

### Step 1: Normalize the decision matrix 

In [15]:
# p_ij = x_ij/sum{j}(x_ij)
df1 = df.copy()
df1 = df1 / Sum
df1

In [16]:
# p_ij*ln(p_ij)
df2 = df1.copy()
df2 = df2 * np.log(df2)
# I am dropping the generosity column since it has a lot of nan values after doing the log 

# for the other columns tat have nan values, I am replacing with 0 since the number is small so it won't have a big affect on the result (the opposite of geneority column)
df2 = df2.fillna(0) 
df2

In [17]:
Sum2 = df2.sum(axis = 0)
Sum2 = Sum2.rename("Sum2") 
Sum2

## Step 2: Compute entropy where m is the number of alternatives

In [18]:
# e_j = -k*p_ij*ln(p_ij)
df3 = -k * Sum2
df3 =df3.rename("Entropy") 
df3

In [19]:
# 1-e_j
df3 = 1 - df3
df3 = pd.DataFrame(df3)
df3

In [20]:
Sum3 = df3.sum(axis=0)
Sum3

### Step 3: Compute the weight vector. These are the objectives of each weight

#### We want to maximize: logged GDP per capita, social support, healthy life expectancy, freedom to make life choices and generosity
    Therefore their weights will be postive
#### We want to minimize: perception of corruption
    Therefore its weight will be negative

In [21]:
# 3- w_j = 1-e_j/sum(1-e_j)
df4 = df3['Entropy'] / Sum3[0]
df4 =df4.rename("Weights")
df4 = pd.DataFrame(df4)
df4.loc["Perceptions of corruption"]= -df4.loc["Perceptions of corruption"]
df4

### Ranking :

In [22]:
df5 = df1.copy()
df5 = df4["Weights"] * df5
df5 = df5.sum(axis = 1)
df5 =df5.rename("Score")
df5 = pd.DataFrame(df5)
df5['Rank'] = df5.rank(axis= 0,ascending= False)
df5 = df5.sort_values(by=['Rank'])
df5

### The Hapiest and Most Unhappy Countries according to entropy

In [23]:
print("The Happiest Country: ", df5.index.values[0]);
print("The Most Unhappy Country: ", df5.index.values[-1]);

In [24]:
happinessFilter = (df5.loc[:,"Rank"] > 141) ;
y = df5.index.values[-8:]
ax = sns.barplot(x = "Score", y = y, data = df5[happinessFilter], palette = "coolwarm");
ax.set_title('The most unhappy countries')

In [25]:
happinessFilter = (df5.loc[:,"Rank"] < 8) ;
y = df5.index.values[0:7]
ax = sns.barplot(x = "Score", y = y, data = df5[happinessFilter], palette = "coolwarm");
ax.set_title('The most happy countries')

In [26]:
fig = px.choropleth(df5, locations = df5.index.values, color = "Score", locationmode='country names')
fig.update_layout(title_text = 'World Happiness Index according to entropy', title_x = 0.5)
fig.show()

## WSM + WPM:

### Step 0:

In [27]:
#Add max for each column
df6 =df.copy()
Max = df6.max(numeric_only=True)
Max = Max.rename('Max')
#Add min for each column
Min = df6.min(numeric_only=True)
Min = Min.rename('Min')
#Add weigths
criteria = len(df6.columns)
Weight = 1/criteria
Weigths = []
for i in range(len(df6.columns)):
     Weigths.append(Weight)
W = pd.Series(Weigths, index = df6.columns)
W = W.rename("Weights")
df6 = df6.append([Max,Min,W], ignore_index=False)
df6

### WSM or Simple Additive Weighting(SAW) method

### Step 1: Normalize the decision matrix

In [28]:
#If the criterion is minimized then the value in the Min row should be considered otherwise it's the opposite
#Normalization: [MAX] p_ij = x_ij/max{j}(x_ij) # [MIN] min{j}{x_ij}/x_ij
df7 =pd.DataFrame()
df7["Logged GDP per capita"] = df6["Logged GDP per capita"] / df6.loc["Min"]["Logged GDP per capita"]
df7["Social support"] = df6["Social support"] / df6.loc["Min"]["Social support"]
df7["Healthy life expectancy"] = df6["Healthy life expectancy"] / df6.loc["Min"]["Healthy life expectancy"]
df7["Freedom to make life choices"] = df6["Freedom to make life choices"] / df6.loc["Min"]["Freedom to make life choices"]
df7["Perceptions of corruption"] = df6["Perceptions of corruption"] / df6.loc["Max"]["Perceptions of corruption"]
df7 = df7.drop(["Max","Min","Weights"], axis =0)
df7

### Step 2: WSM

In [29]:
df8 = df6.loc["Weights"] ** df7
df8 = df8.sum(axis= 1)
df8 =df8.rename("Sum")
df8 = pd.DataFrame(df8)
df8['Rank'] = df8.rank(axis= 0,ascending= False)
df8 = df8.sort_values(by=['Rank'])
df8

### The Hapiest and Most Unhappy Countries according to WSM

In [30]:
print("The Happiest Country: ", df8.index.values[0]);
print("The Most Unhappy Country: ", df8.index.values[-1]);

In [31]:
happinessFilter = (df8.loc[:,"Rank"] > 141) ;
y = df8.index.values[-8:]
ax = sns.barplot(x = "Sum", y = y, data = df8[happinessFilter], palette = "coolwarm");
ax.set_title('The most unhappy countries')

In [32]:
happinessFilter = (df8.loc[:,"Rank"] < 8) ;
y = df8.index.values[0:7]
ax = sns.barplot(x = "Sum", y = y, data = df8[happinessFilter], palette = "coolwarm");
ax.set_title('The most happy countries')

In [33]:
fig = px.choropleth(df8, locations = df8.index.values, color = "Sum", locationmode='country names')
fig.update_layout(title_text = 'World Happiness Index according to WSM', title_x = 0.5)
fig.show()

### Step 3: WPM

In [34]:
df9 = df6.loc["Weights"] * df7
df9 = df9.product(axis= 1)
df9 =df9.rename("Product")
df9 = pd.DataFrame(df9)
df9['Rank'] = df9.rank(axis= 0,ascending= False)
df9 = df9.sort_values(by=['Rank'])
df9

### The Hapiest and Most Unhappy Countries according to WPM

In [35]:
print("The Happiest Country: ", df9.index.values[0]);
print("The Most Unhappy Country: ", df9.index.values[-1]);

In [36]:
happinessFilter = (df9.loc[:,"Rank"] > 141) ;
y = df9.index.values[-8:]
ax = sns.barplot(x = "Product", y = y, data = df9[happinessFilter], palette = "coolwarm");
ax.set_title('The most unhappy countries')

In [37]:
happinessFilter = (df9.loc[:,"Rank"] < 8) ;
y = df9.index.values[0:7]
ax = sns.barplot(x = "Product", y = y, data = df9[happinessFilter], palette = "coolwarm");
ax.set_title('The most happy countries')

In [38]:
fig = px.choropleth(df9, locations = df9.index.values, color = "Product", locationmode='country names')
fig.update_layout(title_text = 'World Happiness Index according to WPM', title_x = 0.5)
fig.show()

### WASPASS:
     We redo the same steps as the previous methods for the 3 first steps

#### Step 4: combine the two methods

In [39]:
Lambda = 0.5
df10 = pd.DataFrame()
df10["WSM"] = df8["Sum"]
df10["WPM"] = df9["Product"]
df10["Score"] = df10["WSM"] + (1-Lambda) * df10["WPM"]
df10['Rank'] = df10["Score"].rank(axis= 0,ascending= False)
df10 = df10.sort_values(by=['Rank'])
df10

### The Hapiest and Most Unhappy Countries according to WASPASS

In [40]:
print("The Happiest Country: ", df10.index.values[0]);
print("The Most Unhappy Country: ", df10.index.values[-1]);

In [41]:
happinessFilter = (df10.loc[:,"Rank"] > 141) ;
y = df10.index.values[-8:]
ax = sns.barplot(x = "Score", y = y, data = df10[happinessFilter], palette = "coolwarm");
ax.set_title('The most unhappy countries')

In [42]:
happinessFilter = (df10.loc[:,"Rank"] < 8) ;
y = df10.index.values[0:7]
ax = sns.barplot(x = "Score", y = y, data = df10[happinessFilter], palette = "coolwarm");
ax.set_title('The most happy countries')

In [43]:
fig = px.choropleth(df10, locations = df10.index.values, color = "Score", locationmode='country names')
fig.update_layout(title_text = 'World Happiness Index according WASPASS', title_x = 0.5)
fig.show()

### Topsis

In [44]:
df11= df.copy() 
# nb criteria
n = df11.shape[1]
#nb alternatives
m = df11.shape[0]
#Add weigths
criteria = len(df11.columns)
Weight = 1/criteria
Weigths = []
for i in range(len(df11.columns)):
     Weigths.append(Weight)
W = pd.Series(Weigths, index = df11.columns)
W = W.rename("Weights")
Sum = df.sum(axis = 0)
Sum = Sum.rename("Sum")
SQT = np.sqrt((np.power(df, 2)).sum(axis=0))
SQT = SQT.rename("SQRT(Sum(x_ij^2))")
df11 = df11.append([W,Sum,SQT], ignore_index=False)
df11

### Step 1 : Normalization

In [45]:
#Normalization: r_ij
df12 = df11 / df11.loc["SQRT(Sum(x_ij^2))"]
df12 = df12.drop(["Weights","Sum","SQRT(Sum(x_ij^2))"],axis=0)
df12

### Step 2 : Weighted normalized decision matrix

In [46]:
#2- v_ij = r_ij*w_ij
df13 = df12 * df11.loc["Weights"]
df13 

### Step 3: Calculate the ideal best

In [47]:
#3- V_j+ and V_j-
df14 = pd.DataFrame()
Max = df13.max()
Max = Max.rename('Max')
#Add min for each column
Min = df13.min()
Min = Min.rename('Min')
df14 = df14.append([Max,Min], ignore_index=False)
#Chose the v_j+ row
v_jPlus = df14.loc["Max"][0:4]
v_jPlus= v_jPlus.append(df14.loc["Min"][4:5])
v_jPlus = v_jPlus.rename("v_j+")
#Chose the v_j- row
v_jMoins = df14.loc["Min"][0:4]
v_jMoins= v_jMoins.append(df14.loc["Max"][4:5])
v_jMoins = v_jMoins.rename("v_j-")
df14 = df14.append([v_jPlus,v_jMoins], ignore_index=False)
df14

### Step 4: Euclidean distance from ideal best and worst

In [48]:
#4.1- SQR(v_ij - v_j+)
df15 = np.power((df13 - df14.loc["v_j+"]),2)
df15["Sum"] = df15.sum(axis=1)
df15["S_i+"] = np.sqrt(df15["Sum"])
df15

In [49]:
#4.2- SQR(v_ij - v_j+)
df16 = np.power((df13 - df14.loc["v_j-"]),2)
df16["Sum"] = df16.sum(axis=1)
df16["S_i-"] = np.sqrt(df16["Sum"])
df16

### Step 4: Calculate the performance score

In [50]:
df17 = pd.DataFrame()
df17["S_i+"] = df15["S_i+"]
df17["S_i-"] = df16["S_i-"]
df17["Sum"] = df17.sum(axis=1)
df17["Pi"] = df17["S_i-"] / df17["Sum"]
df17['Rank'] = df17["Pi"].rank(axis= 0,ascending= False)
df17 = df17.sort_values(by=['Rank'])
df17

### The Hapiest and Most Unhappy Countries according to Topsis

In [51]:
print("The Happiest Country: ", df17.index.values[0]);
print("The Most Unhappy Country: ", df17.index.values[-1]);

In [52]:
happinessFilter = (df17.loc[:,"Rank"] > 141) ;
y = df17.index.values[-8:]
ax = sns.barplot(x = "Pi", y = y, data = df17[happinessFilter], palette = "coolwarm");
ax.set_title('The most unhappy countries')

In [53]:
happinessFilter = (df17.loc[:,"Rank"] < 8) ;
y = df17.index.values[0:7]
ax = sns.barplot(x = "Pi", y = y, data = df17[happinessFilter], palette = "coolwarm");
ax.set_title('The most happy countries')

In [54]:
fig = px.choropleth(df17, locations = df17.index.values, color = "Pi", locationmode='country names')
fig.update_layout(title_text = 'World Happiness Index according to topsis', title_x = 0.5)
fig.show()